In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F



##The encoder in the paper takes x in R^(TxHxW) and y in R^(TxE) and maps them to R^(TxL) the dimensionality of the context encoder
##and encoder is set to 5. So this cnn needs to output (32x5) so each (surface in R^(5x5) -> (z in R^5)

##we enhance the dimensionality of the iv surface first by upgrading the number of channels. The reasoning behind this is similar to why 
##we do this in transformer architecture. A larger dimensional space will be able to capture more nuanced information and represent it in number form
##then we compress this to something digestable
class CNN(nn.Module):
    #input_size and output_size represent the number of channels in the input and output data
    #channels is the number of dimensions a single data point will have ie RGB = 3 channelss
    def __init__(self, input_size, output_size):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(input_size, output_size, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(output_size, output_size, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(output_size, output_size, kernel_size=3, stride=1, padding=1)
        self.fc = nn.Linear(output_size * 5 * 5, 5)

    def forward(self, x):
        batch_size, H, W = x.shape
        x = x.reshape(batch_size, 1, H, W)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        #print(f"x shape before resize and convultion passthrough is {x.shape}")
        #x = x.reshape(batch_size, 1, 5)
        x = x.view(batch_size, -1)
        x = self.fc(x)
        #print(f"after passthrough into convultion layers and fully connected layer ther shape of x is {x.shape}")
        return x

class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, y):
        x = F.relu(self.fc1(y))
        x = self.fc2(y)
        return y

In [2]:
##had to make another cnn for decoder due difference in dimensionality

##in here the output size should be the number of days in the future?

##for now output_size = 1 so we predict 1 day into the future?
class TCNN(nn.Module):
    def __init__(self, input_size, output_size, num_surfaces):
        super(TCNN, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 5 * 5 * num_surfaces)
        self.output_size = output_size
        self.num_surfaces = num_surfaces
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        x = x.view(-1, self.num_surfaces, 5, 5)
        return x

In [3]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size):
        super(Encoder, self).__init__()
        self.cnn = CNN(input_size, 5)
        self.mlp = nn.Identity()
        self.lstm = nn.LSTM(5 + 3, hidden_size, num_layers=2, batch_first=True, dropout=0.2)
        self.linear_mu = nn.Linear(hidden_size, latent_size)
        self.linear_sigma = nn.Linear(hidden_size, latent_size)

    def forward(self, x, y):
        x_encoded = self.cnn(x)
        y_encoded = self.mlp(y)
        y_encoded = torch.squeeze(y_encoded, dim=1)
        #print(f"x_encoded shape is {x_encoded.shape}  y_encoded shape is {y_encoded.shape}")
        encoded = torch.cat((x_encoded, y_encoded), dim=-1)
        #print(f"concatenated vector is of size {encoded.shape}")
        _, (hidden, _) = self.lstm(encoded)
        #print("hidden state created")
        hidden = hidden[-1]  # Take the last hidden state
        mu = self.linear_mu(hidden)
        log_var = self.linear_sigma(hidden)
        z = self.reparameterize(mu, log_var)
        #print("encoding successful")
        return z, mu, log_var

    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + eps * std

class ContextEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, context_size):
        super(ContextEncoder, self).__init__()
        self.cnn = CNN(input_size, 5)
        self.mlp = nn.Identity()
        self.lstm = nn.LSTM(5 + 3, hidden_size, num_layers=2, batch_first=True, dropout=0.2)
        self.linear = nn.Linear(hidden_size, context_size)

    def forward(self, x_c, y_c):
        x_encoded = self.cnn(x_c)
        y_encoded = self.mlp(y_c)
        y_encoded = torch.squeeze(y_encoded, dim=1)
        #print(f"x_encoded size = {x_encoded.shape} ||y_encoded size = {y_encoded.shape}")
        encoded = torch.cat((x_encoded, y_encoded), dim=-1)
        _, (hidden, _) = self.lstm(encoded)
        hidden = hidden[-1]  # Take the last hidden state
        zeta = self.linear(hidden)
        #print("context encoding successful")
        return zeta

class Decoder(nn.Module):
    def __init__(self, latent_size, hidden_size, output_size, num_surfaces):
        super(Decoder, self).__init__()
        self.lstm = nn.LSTM(latent_size + 5, hidden_size, num_layers=2, batch_first=True, dropout=0.2) #latent
        self.tcnn = TCNN(hidden_size, output_size, num_surfaces)
        self.mlp = nn.Linear(hidden_size, 1)

    def forward(self, z, zeta):
        # Reshape z and zeta to have shape (1, 5)
        z = z.view(1, -1)
        zeta = zeta.view(1, -1)
        # Concatenate z and zeta along the second dimension to get shape (1, 10)
        z_concat = torch.cat((z, zeta), dim=1)
        
        #print(f"z_concat is of size {z_concat.shape}")
        hidden, _ = self.lstm(z_concat)
        #print('i am here')
        #print(f"hidden state published, hidden state shape {hidden.shape}")
        x_n = self.tcnn(hidden) 
        #print('i am here')
        r_n = self.mlp(hidden)
        #print("decoding successful")
        return torch.squeeze(x_n, dim=0), torch.squeeze(r_n, dim=0)

In [4]:
class CVAE(nn.Module):
    def __init__(self, input_size, hidden_size, latent_size, context_size, output_size, num_surfaces):
        super(CVAE, self).__init__()
        self.encoder = Encoder(input_size, hidden_size, latent_size)
        self.context_encoder = ContextEncoder(input_size, hidden_size, context_size)
        self.decoder = Decoder(latent_size, hidden_size, output_size, num_surfaces)
        self.latent_size = latent_size
        self.context_size = context_size

    def forward(self, x, y, x_c, y_c):
        #print('1')
        z, mu, log_var = self.encoder(x, y) ##we have sampled z from distribution here
        #print(f"The shape of latent representation z is {z.shape}")
        #print('2')
        zeta = self.context_encoder(x_c, y_c) ##we have sampled zeta from distribution here
        #print(f"the shape of zeta (encoded context) is {zeta.shape}")
        x_n, r_n = self.decoder(z, zeta)
        return x_n, r_n, mu, log_var

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Load the DataFrame from the provided CSV file
file_path = 'combined_iv_data.csv'
df = pd.read_csv(file_path, header=[0, 1], index_col=0)

# Extract extra features
Y = df[['Log Return', 'Skew', 'Slope']]

# Extract IV surface data
X = df.drop(['Log Return', 'Skew', 'Slope'], axis=1)

# Normalize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
Y_scaled = scaler.fit_transform(Y)

# Split data into training and temporary set
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, Y_scaled, test_size=2000, random_state=42)

# Split the temporary set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

/var/folders/sl/_44jn7jx03b17y35qvlp17sr0000gn/T/ipykernel_75151/3904911613.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X = df.drop(['Log Return', 'Skew', 'Slope'], axis=1)


In [6]:
import random

def generate_random_day(n):
    """
    Generates a random number between 1 and n
    """
    # Generate a random number between 1 and n
    random_number = random.randint(1, n - 1)
    return random_number

In [8]:
pip install torch_dnnl

ERROR: Could not find a version that satisfies the requirement torch_dnnl (from versions: none)
ERROR: No matching distribution found for torch_dnnl
Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

# Hyperparameters
input_size = 25
hidden_size = 100
latent_size = 5
context_size = 5
input_size = 1
output_size = 5
learning_rate = 1e-3
num_epochs = 500
batch_size = 32
alpha = 1.0
beta = 1e-5


# Reshape the input data to the desired 5-dimensional shape
batch_size_train = X_train.shape[0]
batch_size_val = X_val.shape[0]
batch_size_test = X_test.shape[0]
sequence_length = 1  # 
num_surfaces = 1  # the number of surfaces to generate


H = 5  # Height of the IV surface grid
W = 5  # Width of the IV surface grid

#print(X_train.shape)
X_train_reshaped = X_train.reshape(batch_size_train,  H, W)
X_val_reshaped = X_val.reshape(batch_size_val, H, W)
X_test_reshaped = X_test.reshape(batch_size_test, H, W)
#print(X_train_reshaped.shape)
extra_features_size = 3

y_train_reshaped = y_train.reshape(y_train.shape[0], 1, extra_features_size)
y_val_reshaped = y_val.reshape(y_val.shape[0], 1, extra_features_size)
y_test_reshaped = y_test.reshape(y_test.shape[0], 1, extra_features_size)

train_data = TensorDataset(torch.Tensor(X_train_reshaped), torch.Tensor(y_train_reshaped))
val_data = TensorDataset(torch.Tensor(X_val_reshaped), torch.Tensor(y_val_reshaped))
test_data = TensorDataset(torch.Tensor(X_test_reshaped), torch.Tensor(y_test_reshaped))

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Create the model
model = CVAE(input_size, hidden_size, latent_size, context_size, input_size, num_surfaces)

def loss_function(x_n, x_n_recon, r_n, r_n_recon, mu, log_var):
    # Ensure x_n and r_n are unsqueezed if needed (depends on data shape handling in other parts of your code)
    x_n = torch.unsqueeze(x_n, dim=0) if len(x_n.shape) < 3 else x_n
    r_n = torch.unsqueeze(r_n, dim=0) if len(r_n.shape) < 3 else r_n

    # Calculate the mean squared error, normalized by the number of elements (H*W)
    recon_loss_x = F.mse_loss(x_n_recon, x_n, reduction='mean') / (H * W)
    recon_loss_r = F.mse_loss(r_n_recon, r_n, reduction='mean')

    # Calculate the Kullback-Leibler divergence
    kl_loss = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())

    # Combine the losses with the scaling factors
    total_loss = recon_loss_x + alpha * recon_loss_r + beta * kl_loss

    return total_loss

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
best_val_loss = float('inf')
for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    for batch in train_loader:
        x, y = batch
        batch_size = x.shape[0] #batch_size, context_length, num_surfaces, H, W = x.shape
        
        # Generate a random day t to split the sequence into context and future
        t = batch_size - num_surfaces - 1
        #print(t)

        x_c, x_n = x[:t, :], x[t:, :]
        y_c, y_n = y[:t, :], y[t:, :]
            
        r_n = y_n[:, :, 0]
        
        x_n_recon, r_n_recon, mu, log_var = model(x, y, x_c, y_c)
        
        # Compute the loss
        loss = loss_function(x_n[0], x_n_recon, r_n[0], r_n_recon, mu, log_var)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
    
    # Validation
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch in val_loader:
            x, y = batch
            batch_size = x.shape[0]  # Number of days in the sequence
            
            # Generate a random day t to split the sequence into context and future
            #t = generate_random_day(batch_size)
            t = batch_size - num_surfaces - 1
            
            x_c, x_n = x[:t, :], x[t:, :]
            y_c, y_n = y[:t, :], y[t:, :]
            
            # Extract the log return feature from y_n
            r_n = y_n[:, :, 0]
            
            x_n_recon, r_n_recon, mu, log_var = model(x, y, x_c, y_c)
            #print(f"x_n_recon: {x_n_recon}")
            
            # Compute the loss
            loss = loss_function(x_n[0], x_n_recon, r_n[0], r_n_recon, mu, log_var)
            
            val_loss += loss.item()
    
    # Print the losses for each epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader):.4f}, Val Loss: {val_loss/len(val_loader):.4f}")
    
    # Save the best model based on validation loss
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

# Testing
model.load_state_dict(torch.load('best_model.pth'))
model.eval()
test_loss = 0.0
with torch.no_grad():
    for batch in test_loader:
        x, y = batch
        batch_size = x.shape[0]  # Number of days in the sequence
        
        # Generate a random day t to split the sequence into context and future
        t = batch_size - num_surfaces - 1
        
        x_c, x_n = x[:, :t], x[:, t:]
        y_c, y_n = y[:, :t], y[:, t:]
        
        # Extract the log return feature from y_n
        r_n = y_n[:, :, 0]
        
        x_n_recon, r_n_recon, mu, log_var = model(x, y, x_c, y_c)
        
        # Compute the loss
        loss = loss_function(x_n[0], x_n_recon, r_n[0], r_n_recon, mu, log_var)
        
        test_loss += loss.item()

print(f"Test Loss: {test_loss/len(test_loader):.4f}") 

/var/folders/sl/_44jn7jx03b17y35qvlp17sr0000gn/T/ipykernel_75151/1260078627.py:59: UserWarning: Using a target size (torch.Size([1, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  recon_loss_r = F.mse_loss(r_n_recon, r_n, reduction='mean')


Epoch [1/500], Train Loss: 1.0306, Val Loss: 1.0999
Epoch [2/500], Train Loss: 0.8024, Val Loss: 1.0828
Epoch [3/500], Train Loss: 0.6992, Val Loss: 0.8012
Epoch [4/500], Train Loss: 0.5023, Val Loss: 0.3835
Epoch [5/500], Train Loss: 0.2391, Val Loss: 0.2193
Epoch [6/500], Train Loss: 0.1572, Val Loss: 0.2410
Epoch [7/500], Train Loss: 0.1415, Val Loss: 0.1208
Epoch [8/500], Train Loss: 0.1221, Val Loss: 0.0818
Epoch [9/500], Train Loss: 0.0918, Val Loss: 0.1231
Epoch [10/500], Train Loss: 0.1166, Val Loss: 0.1708
Epoch [11/500], Train Loss: 0.1445, Val Loss: 0.1065
Epoch [12/500], Train Loss: 0.0927, Val Loss: 0.1217
Epoch [13/500], Train Loss: 0.0833, Val Loss: 0.1114
Epoch [14/500], Train Loss: 0.0922, Val Loss: 0.0848
Epoch [15/500], Train Loss: 0.1320, Val Loss: 0.1056
Epoch [16/500], Train Loss: 0.0867, Val Loss: 0.1424
Epoch [17/500], Train Loss: 0.0804, Val Loss: 0.0719
Epoch [18/500], Train Loss: 0.0625, Val Loss: 0.0724
Epoch [19/500], Train Loss: 0.0985, Val Loss: 0.0600
Ep

/var/folders/sl/_44jn7jx03b17y35qvlp17sr0000gn/T/ipykernel_75151/1260078627.py:58: UserWarning: Using a target size (torch.Size([1, 0, 5])) that is different to the input size (torch.Size([1, 5, 5])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  recon_loss_x = F.mse_loss(x_n_recon, x_n, reduction='mean') / (H * W)


RuntimeError: The size of tensor a (5) must match the size of tensor b (0) at non-singleton dimension 1